In [1]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=4182b0bcffb90966e20c152cef05f600ed9e3a476b0a70fff12ac88c62026910
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [0]:
from tqdm import tqdm
import os 

import numpy as np
import pandas as pd
import datetime

from sklearn.preprocessing import MinMaxScaler

import yfinance as yf

from sklearn.linear_model import LinearRegression

In [0]:
os.mkdir('data')

In [46]:
def get_close_stock_price(codes, start_date='2014-01-01', end_date='2019-04-30'):
    data = { code : yf.download(code, start_date, end_date) for code in codes }
    data = { key : pd.DataFrame(value, columns=['Close']).rename(columns={'Close': f'{key}'}) for key, value in data.items() }
    df = None
    for key, value in data.items():
        if df is None:
            df = value
        else:
            df = df.merge(value, left_index=True, right_index=True)
    return df

df = get_close_stock_price(['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'FB'])
df.to_csv('data/close_price_big_5.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [47]:
df = pd.read_csv('data/close_price_big_5.csv', index_col=['Date'], infer_datetime_format=True, parse_dates=['Date'])
df.head()

,AAPL,GOOGL,MSFT,AMZN,FB
Date,,,,,
2014-01-02,79.018570,557.117126,37.160000,397.970001,54.709999
2014-01-03,77.282860,553.053040,36.910000,396.440002,54.560001
2014-01-06,77.704285,559.219238,36.130001,393.630005,57.200001
2014-01-07,77.148575,570.000000,36.410000,398.029999,57.919998
2014-01-08,77.637146,571.186157,35.759998,401.920013,58.230000


In [0]:
data = df.to_numpy()

In [13]:
data.shape

(1592, 5)

In [0]:

len_train = len(data) * 7 //10
training_data = data[:len_train]
valid_data = data[len_train:]
scaler = MinMaxScaler(feature_range=(0, 1))
scale = scaler.fit(training_data)
d = np.concatenate((scaler.transform(training_data) , scaler.transform(valid_data)))


In [0]:

def create_Xy(data, history=1):
    '''df has index as date
    columns are stock price of the companies
    '''
    X, y, dates = [], [], []
    for row_idx in range(history, len(df)):
        y.append( data[row_idx,:].reshape(-1) )
        X.append( data[row_idx-history:row_idx,:].reshape(-1) )
        dates.append(df.index[row_idx])
        
    # print(x,y)
    X, y = np.array(X), np.array(y)
    return X, y, dates

X,y,dates = create_Xy(data, history=2)
# use 7/10 of the dataset to train (use past price to predict future price)
n_train = len(y) * 7 // 10
X_train, y_train, dates_train, X_test, y_test, dates_test = X[:n_train], y[:n_train], dates[:n_train], X[n_train:], y[n_train:], dates[n_train:]

In [0]:
# te = scale.inverse_transform()
relation_error = np.mean((abs(scale.inverse_transform(ypred)[:,0]-scale.inverse_transform(y_test)[:,0])/scale.inverse_transform(y_test)[:,0]))
relation_error

0.018442380845631292

#Arima


In [0]:
!pip install pyramid-arima

     |████████████████████████████████| 604kB 2.7MB/s 


In [0]:
from pyramid.arima import auto_arima

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr

In [0]:
data = df
train_len = int(len(data) *70/100)
train = data[:train_len]
valid = data[train_len:]

training = train['AAPL']
validation = valid['AAPL']

model = auto_arima(training, start_p=1, start_q=1,max_p=3, max_q=3, m=12,start_P=0, seasonal=True,d=1, D=1, trace=True,error_action='ignore',suppress_warnings=True)
model.fit(training)

forecast = model.predict(n_periods=len(validation))
# 

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1183.855, BIC=1200.823, Fit time=1.798 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1279.982, BIC=1286.769, Fit time=0.030 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1227.568, BIC=1241.142, Fit time=0.598 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1182.633, BIC=1196.208, Fit time=0.853 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1184.511, BIC=1201.479, Fit time=1.244 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1281.951, BIC=1292.132, Fit time=0.136 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=1184.517, BIC=1201.485, Fit time=2.845 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=1184.037, BIC=1204.399, Fit time=8.895 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=1181.401, BIC=1191.582, Fit time=0.510 seconds
Fit ARIMA: order=(0, 1, 0) s

In [0]:
forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

In [0]:
forecast['Prediction']

Date
2019-12-04    259.887734
2019-12-05    260.642921
2019-12-06    262.340213
2019-12-09    261.523745
2019-12-10    261.504432
                 ...    
2020-04-23    323.393521
2020-04-24    324.269543
2020-04-27    326.087670
2020-04-28    325.392036
2020-04-29    325.493558
Name: Prediction, Length: 101, dtype: float64

In [0]:
relation_error = np.mean((abs(validation-forecast['Prediction'])/validation))
relation_error

0.11989478629158211

#Prophet

In [0]:
!pip install fbprophet

In [0]:
#importing prophet
from fbprophet import Prophet

#creating dataframe
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

for i in range(0,len(data)):
    new_data['Date'][i] = df.index[i]
    new_data['Close'][i] = df['AAPL'][i]

new_data['Date'] = pd.to_datetime(new_data.Date,format='%Y-%m-%d')
new_data.index = new_data['Date']

#preparing data
new_data.rename(columns={'Close': 'y', 'Date': 'ds'}, inplace=True)

#train and validation
train_len = int(len(data) *70/100)
train = new_data[:train_len]
valid = new_data[train_len:]

#fit the model
model = Prophet()
model.fit(train)

#predictions
close_prices = model.make_future_dataframe(periods=len(valid))
forecast = model.predict(close_prices)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Date
2019-12-04    261.74
2019-12-05    265.58
2019-12-06    270.71
2019-12-09    266.92
2019-12-10    268.48
               ...  
2020-04-23    275.03
2020-04-24    282.97
2020-04-27    283.17
2020-04-28    278.58
2020-04-29    287.73
Name: y, Length: 101, dtype: object

In [0]:
predict = list(forecast['yhat'][train_len:])
y_true = list(valid['y'])

In [0]:
res = [abs(y_true[i]-predict[i])/y_true[i] for i in range(len(y_true))]
relation_error = np.mean(res)
relation_error

0.13241051409986276

#Linear

In [0]:
from sklearn.metrics import mean_squared_error

In [156]:
reg = LinearRegression().fit(X_train, y_train[:,0])
ypred = reg.predict(X_test)
relation_error = np.mean((abs(ypred-y_test[:,0])/y_test[:,0]))
relation_error

0.011854602769182864

In [136]:
reg = LinearRegression().fit(X_train, y_train[:0])
ypred = reg.predict(X_test)
relation_error = np.mean((abs(ypred-y_test[:,0])/y_test[:,0]))
relation_error

0.0130856305923329

In [163]:
for i in range(5):
  reg = LinearRegression().fit(X_train, y_train[:,i])
  ypred = reg.predict(X_test)
  relation_error = np.mean((abs(ypred-y_test[:,i])/y_test[:,i]))
  print(f'{df.columns[i]} - relationerror:{relation_error} - mse:{mean_squared_error(ypred,y_test[:,i])}')
  

AAPL - relationerror:0.011854602769182864 - mse:9.510824936531966
GOOGL - relationerror:0.0130856305923329 - mse:384.7006012277215
MSFT - relationerror:0.011219631041094885 - mse:2.5732602004452128
AMZN - relationerror:0.014684557996456394 - mse:1102.1571247452323
FB - relationerror:0.018300826846169724 - mse:19.223882510179603


In [0]:
reg_0 = LinearRegression()

#NN

In [0]:
from sklearn.neural_network import MLPRegressor

In [0]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'solver' : ('lbfgs', 'sgd', 'adam'), 'alpha':[0.0001, 0.001],
              'learning_rate' : ('constant', 'invscaling', 'adaptive'),
              }

mlp = MLPRegressor(random_state=1, max_iter=1000).fit(X_train, y_train)
reg = GridSearchCV(mlp, parameters)
reg.fit(X_train, y_train)


# sorted(clf.cv_results_.keys())

In [56]:
h

3

In [0]:
regr = MLPRegressor(random_state=1, max_iter=10000).fit(X_train, y_train)

In [95]:
y_predict = regr.predict(X_test)
relation_error = np.mean((abs(y_predict[:,0]-y_test[:,0])/y_test[:,0]))
relation_error

0.05459959908418759

In [96]:
y_predict = regr.predict(X_test)
relation_error = np.mean((abs(y_predict[:,1]-y_test[:,1])/y_test[:,1]))
relation_error

0.06010928522600052

#BayesianRidge

In [0]:
from sklearn import linear_model

In [0]:
parameters = { 'alpha_1':[1e-5, 1e-7],
              'alpha_2':[1e-5, 1e-7],'lambda_1':[1e-5, 1e-7],
              'lambda_2':[1e-5, 1e-7]}
model = linear_model.BayesianRidge(n_iter=1000)

In [0]:
reg_1 = GridSearchCV(model, parameters)

In [165]:
reg =  GridSearchCV(model, parameters).fit(X_train,y_train[:,0])
y_predict = reg.predict(X_test)
relation_error = np.mean((abs(y_predict-y_test[:,0])/y_test[:,0]))
relation_error

0.011848462268657002

In [167]:
for i in range(5):
  reg.fit(X_train,y_train[:,i])
  y_predict = reg.predict(X_test)
  relation_error = np.mean((abs(y_predict-y_test[:,i])/y_test[:,i]))
  print(f'{df.columns[i]} - relationerror:{relation_error} - mse:{mean_squared_error(y_predict,y_test[:,i])}')

AAPL - relationerror:0.011848462268657002 - mse:9.492179196055691
GOOGL - relationerror:0.013156206236619825 - mse:387.74484819845105
MSFT - relationerror:0.011219220204482303 - mse:2.5683535526854273
AMZN - relationerror:0.014666368803333421 - mse:1099.544466179718
FB - relationerror:0.01842586739000763 - mse:19.381146720107644


In [114]:
reg =  GridSearchCV(model, parameters).fit(X_train,y_train[:,1])
y_predict = reg.predict(X_test)
relation_error = np.mean((abs(y_predict-y_test[:,1])/y_test[:,1]))
relation_error

0.013156206236619825

#Kernel Ridge

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
from sklearn.kernel_ridge import KernelRidge

In [0]:
parameters = { 'alpha':[0, 2],
              'degree':[1,4],'coef0':[1, 4]}
model = KernelRidge()


In [0]:
reg_2 = GridSearchCV(model, parameters)

In [174]:
for i in range(5):
  reg_2.fit(X_train,y_train[:,i])
  y_predict = reg.predict(X_test)
  relation_error = np.mean((abs(y_predict-y_test[:,i])/y_test[:,i]))
  print(f'{df.columns[i]} - relationerror:{relation_error} - mse:{mean_squared_error(y_predict,y_test[:,i])}')

AAPL - relationerror:0.011861750792939442 - mse:9.524182584500863
GOOGL - relationerror:0.8357462334586675 - mse:867222.3836927728
MSFT - relationerror:0.8409887248140229 - mse:7104.174175965652
AMZN - relationerror:0.8802879712061643 - mse:1963458.733284881
FB - relationerror:0.1258548957885862 - mse:787.0519151095449


In [175]:
reg =  GridSearchCV(model, parameters).fit(X_train,y_train[:,0])
y_predict = reg.predict(X_test)
relation_error = np.mean((abs(y_predict-y_test[:,0])/y_test[:,0]))
relation_error

0.011861750792939442

In [176]:
# reg =  GridSearchCV(model, parameters).fit(X_train,y_train[:,1])
reg.fit(X_train,y_train[:,1])
y_predict = reg.predict(X_test)
relation_error = np.mean((abs(y_predict-y_test[:,1])/y_test[:,1]))
relation_error

0.012779095657906855

#Ensemble

In [0]:
from sklearn.ensemble import VotingRegressor

In [178]:
for i in range(5):
  voting = VotingRegressor(estimators=[('1', reg_1),('2', reg_2), ('0', reg_0)]).fit(X_train,y_train[:,i])
  y_predict = voting.predict(X_test)
  relation_error = np.mean((abs(y_predict-y_test[:,i])/y_test[:,i]))
  print(f'{df.columns[i]} - relationerror:{relation_error} - mse:{mean_squared_error(y_predict,y_test[:,i])}')

AAPL - relationerror:0.01185081701224561 - mse:9.511410595402126
GOOGL - relationerror:0.012868444832824172 - mse:374.0526396030915
MSFT - relationerror:0.011210385187269023 - mse:2.575071778817791
AMZN - relationerror:0.015052413181854692 - mse:1130.3678451358503
FB - relationerror:0.018202318722988768 - mse:19.078535872380538


In [92]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)


clf.best_params_

{'C': 1, 'kernel': 'linear'}